In [9]:
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split

file_name = [(x) for x in os.listdir('..\\datasets\\preprocessed_images')]
images = [os.path.join('..\\datasets\\preprocessed_images', x) for x in file_name]
annotations = [os.path.join('..\\datasets\\labels', x) for x in os.listdir('..\\datasets\\labels') if x.endswith(".txt")]

file_name.sort()
images.sort()
annotations.sort()

studies_list = [(x.split(os.sep)[-1].split("_")[0]) for x in images]
studies = list(set(studies_list))

patients = dict()
for study in studies:
    patients[study] = {'0':0,'1':0,'2':0, '3':0}

sum_labels = [0,0,0,0]


for x in range(len(annotations)):
    file = open(annotations[x], "r")
    content = file.read().split(" ")[0]
    key = file_name[x].split("_")[0]
    patients[key][content] = patients[key][content] + 1
    sum_labels[int(content)] = sum_labels[int(content)] + 1
    

print(patients)
print(len(patients))
print(sum_labels)
print(sum(sum_labels))


{'PATIENT813103': {'0': 0, '1': 0, '2': 0, '3': 4}, 'PATIENT957695': {'0': 1, '1': 0, '2': 1, '3': 0}, 'PATIENT363030': {'0': 0, '1': 0, '2': 1, '3': 3}, 'PATIENT809540': {'0': 0, '1': 2, '2': 2, '3': 1}, 'PATIENT897638': {'0': 0, '1': 0, '2': 3, '3': 0}, 'PATIENT1882574': {'0': 1, '1': 1, '2': 2, '3': 1}, 'PATIENT484744': {'0': 0, '1': 0, '2': 1, '3': 3}, 'PATIENT1758859': {'0': 0, '1': 0, '2': 2, '3': 3}, 'PATIENT657573': {'0': 2, '1': 0, '2': 2, '3': 1}, 'PATIENT875329': {'0': 0, '1': 1, '2': 2, '3': 2}, 'PATIENT834176': {'0': 1, '1': 1, '2': 2, '3': 1}, 'PATIENT1499224': {'0': 1, '1': 0, '2': 2, '3': 2}, 'PATIENT624340': {'0': 2, '1': 0, '2': 2, '3': 1}, 'PATIENT190704': {'0': 2, '1': 0, '2': 1, '3': 0}, 'PATIENT1068298': {'0': 2, '1': 0, '2': 2, '3': 1}, 'PATIENT275341': {'0': 1, '1': 1, '2': 3, '3': 0}, 'PATIENT787443': {'0': 2, '1': 0, '2': 1, '3': 1}, 'PATIENT1589236': {'0': 2, '1': 0, '2': 2, '3': 0}, 'PATIENT636571': {'0': 0, '1': 0, '2': 1, '3': 1}, 'PATIENT1380121': {'0': 1

In [10]:
print(0.8 * 84, 0.2 * 84)

print(0.8 * 40, 0.2 * 40)

print(0.8 * 131, 0.2 * 131)

print(0.8 * 100, 0.2 * 100)



67.2 16.8
32.0 8.0
104.80000000000001 26.200000000000003
80.0 20.0


In [11]:
import numpy as np
import random

def custom_test_split(patients, studies, split):
    studies_test = []
    max_sum = 0
    max_sum_list = 0
    max_studies_test = 0

    for _ in range(1000):
        sum_list = np.array([0,0,0,0])
        studies_test = []
        random.shuffle(studies)
        for i in range(len(patients)):
            sum_list = sum_list + np.array([patients[studies[i]]['0'], patients[studies[i]]['1'], patients[studies[i]]['2'], patients[studies[i]]['3']])
            if (sum_list[0] > split[0] or sum_list[1] > split[1] or sum_list[2] > split[2] or sum_list[3] > split[3]):
                sum_list = sum_list - np.array([patients[studies[i]]['0'], patients[studies[i]]['1'], patients[studies[i]]['2'], patients[studies[i]]['3']])
            else:
                studies_test.append(studies[i])
        if sum(sum_list) > max_sum:
            max_sum = sum(sum_list)
            max_sum_list = sum_list
            max_studies_test = studies_test
    print(max_sum)
    print(max_sum_list)
    return max_studies_test
    
test_split = custom_test_split(patients, studies, [17,8,26,20])
print(test_split)

71
[17  8 26 20]
['PATIENT875329', 'PATIENT1589236', 'PATIENT1377872', 'PATIENT733048', 'PATIENT546144', 'PATIENT190704', 'PATIENT813103', 'PATIENT1103487', 'PATIENT498340', 'PATIENT800462', 'PATIENT809540', 'PATIENT953530', 'PATIENT578344', 'PATIENT857167', 'PATIENT823542', 'PATIENT1718935', 'PATIENT346270']


In [12]:
train_split = set(studies).difference(set(test_split))
print(len(train_split), len(test_split))

train_imgs = []
test_imgs = []
train_ann = []
test_ann = []

for i in range(len(images)):
    if images[i].split(os.sep)[-1].split("_")[0] in train_split:
        train_imgs.append(images[i])
    if annotations[i].split(os.sep)[-1].split("_")[0] in train_split:
        train_ann.append(annotations[i])
    if images[i].split(os.sep)[-1].split("_")[0] in test_split:
        test_imgs.append(images[i])
    if annotations[i].split(os.sep)[-1].split("_")[0] in test_split:
        test_ann.append(annotations[i])



68 17


In [13]:
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder) 
            shutil.copy2(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_imgs, '../datasets/train/images' )
move_files_to_folder(train_ann, '../datasets/train/labels')

move_files_to_folder(test_imgs, '../datasets/test/images' )
move_files_to_folder(test_ann, '../datasets/test/labels')

Data Imbalance

In [2]:
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split

file_name = [(x) for x in os.listdir('..\\datasets\\train\\images')]
images = [os.path.join('..\\datasets\\train\\images', x) for x in file_name]
annotations = [os.path.join('..\\datasets\\train\\labels', x) for x in os.listdir('..\\datasets\\train\\labels') if x.endswith(".txt")]

file_name.sort()
images.sort()
annotations.sort()

studies_list = [(x.split(os.sep)[-1].split("_")[0]) for x in images]
studies = list(set(studies_list))

patients = dict()
for study in studies:
    patients[study] = {'0':0,'1':0,'2':0, '3':0}

sum_labels = [0,0,0,0]

files = {'0':[],'1':[],'2':[], '3':[]}
for x in range(len(annotations)):
    file = open(annotations[x], "r")
    content = file.read().split(" ")[0]
    key = file_name[x].split("_")[0]
    files[content].append((key + ".png"))
    patients[key][content] = patients[key][content] + 1
    sum_labels[int(content)] = sum_labels[int(content)] + 1
    

print(patients)
print(len(patients))
print(sum_labels)
print(sum(sum_labels))


{'PATIENT659743': {'0': 3, '1': 0, '2': 1, '3': 0}, 'PATIENT262899': {'0': 1, '1': 1, '2': 1, '3': 2}, 'PATIENT624340': {'0': 2, '1': 0, '2': 2, '3': 1}, 'PATIENT1376694': {'0': 1, '1': 0, '2': 2, '3': 0}, 'PATIENT1508869': {'0': 1, '1': 0, '2': 1, '3': 2}, 'PATIENT275341': {'0': 1, '1': 1, '2': 3, '3': 0}, 'PATIENT1882574': {'0': 1, '1': 1, '2': 2, '3': 1}, 'PATIENT1845131': {'0': 2, '1': 0, '2': 0, '3': 2}, 'PATIENT897638': {'0': 0, '1': 0, '2': 3, '3': 0}, 'PATIENT636571': {'0': 0, '1': 0, '2': 1, '3': 1}, 'PATIENT483178': {'0': 0, '1': 1, '2': 3, '3': 1}, 'PATIENT935034': {'0': 1, '1': 0, '2': 3, '3': 1}, 'PATIENT957695': {'0': 1, '1': 0, '2': 1, '3': 0}, 'PATIENT1791107': {'0': 0, '1': 1, '2': 4, '3': 0}, 'PATIENT865639': {'0': 0, '1': 0, '2': 2, '3': 3}, 'PATIENT994827': {'0': 1, '1': 0, '2': 1, '3': 2}, 'PATIENT71049': {'0': 0, '1': 3, '2': 2, '3': 0}, 'PATIENT1535730': {'0': 1, '1': 0, '2': 1, '3': 1}, 'PATIENT1401612': {'0': 0, '1': 0, '2': 2, '3': 3}, 'PATIENT1853266': {'0': 

In [4]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
# Importing the dataset
# Read dataset

new_0 = resample(files['0'], 
                                 replace=True,     
                                 n_samples=120)
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder) 
            shutil.copy2(f"prep", destination_folder)
        except:
            print(f)
            assert False

['PATIENT614950.png', 'PATIENT1653957.png', 'PATIENT844769.png', 'PATIENT257654.png', 'PATIENT1845131.png', 'PATIENT682110.png', 'PATIENT1653957.png', 'PATIENT844769.png', 'PATIENT257654.png', 'PATIENT624340.png', 'PATIENT614950.png', 'PATIENT659743.png', 'PATIENT1736897.png', 'PATIENT1423834.png', 'PATIENT682110.png', 'PATIENT1738359.png', 'PATIENT659743.png', 'PATIENT624340.png', 'PATIENT596080.png', 'PATIENT1464627.png', 'PATIENT834176.png', 'PATIENT1376694.png', 'PATIENT899191.png', 'PATIENT674740.png', 'PATIENT1380121.png', 'PATIENT682110.png', 'PATIENT1068298.png', 'PATIENT844769.png', 'PATIENT614950.png', 'PATIENT262899.png', 'PATIENT682110.png', 'PATIENT1068298.png', 'PATIENT657573.png', 'PATIENT852711.png', 'PATIENT1845131.png', 'PATIENT624340.png', 'PATIENT935034.png', 'PATIENT694785.png', 'PATIENT1845131.png', 'PATIENT257654.png', 'PATIENT1517819.png', 'PATIENT1738359.png', 'PATIENT852711.png', 'PATIENT1853266.png', 'PATIENT834176.png', 'PATIENT1068298.png', 'PATIENT1380121.

# Creating CSV file for Classification training

In [9]:
import shutil
import os
import pandas as pd
from sklearn.model_selection import train_test_split

train_images = [os.path.join('..\\datasets\\train\\images', x) for x in os.listdir('..\\datasets\\train\\images') if x.endswith(".png")]
train_annotations = [os.path.join('..\\datasets\\train\\labels', x) for x in os.listdir('..\\datasets\\train\\labels') if x.endswith(".txt")]

test_images = [os.path.join('..\\datasets\\test\\images', x) for x in os.listdir('..\\datasets\\test\\images') if x.endswith(".png")]
test_annotations = [os.path.join('..\\datasets\\test\\labels', x) for x in os.listdir('..\\datasets\\test\\labels') if x.endswith(".txt")]

images = train_images + test_images
annotations = train_annotations + test_annotations

images.sort()
annotations.sort()



dataset = pd.DataFrame(columns=['ID', 'Path', 'Label', 'Split'])

for i in range(len(images)):

    dataset.loc[i,'ID'] = images[i].split(os.sep)[-1].split('.')[0]
    dataset.loc[i,'Path'] = images[i]
    file = open(annotations[i], "r")
    dataset.loc[i,'Label'] = file.read().split(" ")[0]
    dataset.loc[i,'Split'] = images[i].split(os.sep)[-3]
dataset.to_csv('dataset.csv', index = 0)


In [93]:
targets = []
weights = []
train_images = []
for key in files.keys():
    print(len(files[key]), 1/len(files[key]))
    key_weight = 1/len(files[key])
    for image in files[key]:
        weights.append(key_weight)
        targets.append(int(key))
        train_images.append(image)

67 0.014925373134328358
32 0.03125
105 0.009523809523809525
80 0.0125


In [97]:
from torch.utils.data import Dataset
from sklearn.utils import shuffle
from torch.utils.data import DataLoader
import torch.utils.data
import numpy as np

weights = torch.DoubleTensor(weights)
train = torch.IntTensor(np.array(range(284)))
targets = torch.IntTensor(targets)

train_dataset = torch.utils.data.TensorDataset(train, targets)

sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights), replacement=True)
trainloader = DataLoader(train_dataset, batch_size=284, sampler=sampler,
                         shuffle=False,
                         num_workers=0, pin_memory=False)

for batch in trainloader:
    print("a")
    print(batch[0])


a
tensor([153, 254, 136,  56,  63, 259,  75, 261, 185,  76,  63,  84,  79,  93,
         52, 273, 116, 272,  19, 199, 273,  25,  57, 185, 238, 204, 269,  14,
        127, 242,  73,  93,  34, 210,   6,  86,  88, 230, 154, 239,  85,  93,
        126,  69,  83, 149,  48,  67, 113, 274,  70, 278, 266, 105,  92,  92,
        236,  41,  89, 276, 218,  69,  76, 145,  78, 210,  77, 143, 222, 160,
         34,  76,  80, 138, 271, 204,  60,  94, 130, 184,  61, 275, 258,  94,
         71,  20, 183,  61, 205,  24, 211, 220,  93, 191,  93,  90,  31,  62,
         16, 192, 110, 249,  32,  78, 205, 250,  76, 116,  69, 251, 165,  44,
        209, 235, 278,  53,   9,  66, 235, 282, 262,  25,   5,  22,  40, 244,
         85,  82,  31, 241, 135,  51, 115,  32, 227,  84, 177, 248, 190, 269,
        215, 186,  81,  70, 215,  87, 167,  82, 109,  68, 212,  34,  94,  98,
        242, 109, 196,  94, 258,  94,  51,  74, 164,  73, 243, 274,  96,  17,
        180,  18, 100, 152,  66,  56,  91,  14,  76, 277,  74,